In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
from bs4 import BeautifulSoup
import regex as re

# from https://wiki.openstreetmap.org/wiki/Key:maxspeed#Implicit_maxspeed_values
# better table at https://wiki.openstreetmap.org/wiki/Default_speed_limits

html = '''<tbody><tr><td rowspan="3"><b>AR</b></td><td>AR:urban</td><td>40</td><td rowspan="3">AR:rural</td><td rowspan="3">110</td><td rowspan="3"></td><td rowspan="3"></td><td rowspan="3">—</td></tr><tr><td>AR:urban:primary</td><td>60</td></tr><tr><td>AR:urban:secondary</td><td>60</td></tr><tr><td rowspan="3"><b>AT</b></td><td rowspan="3">AT:urban</td><td rowspan="3">50</td><td rowspan="3">AT:rural</td><td rowspan="3">100</td><td>AT:bicycle_road</td><td>30</td><td rowspan="3">—</td></tr><tr><td>AT:trunk</td><td>100</td></tr><tr><td>AT:motorway</td><td>130</td></tr><tr><td><b>AU</b></td><td>AU:urban</td><td>50 or 60 in NT<p>(Northern Territory)</p></td><td>AU:rural</td><td>100 or 110 in NT &amp; WA<p>(Northern Territory &amp; Western Australia)</p></td><td></td><td></td><td><ul><li>Default (unsigned) speed limit in built-up areas is 50km/h in all states and mainland territories excluding the Northern Territory, where it is 60km/h.</li><li>Default (unsigned) speed limit outside of urban built-up areas (such as suburbs or towns) is 100km/h in all states and mainland territories excluding the Northern Territory and Western Australia, where it is 110km/h.</li><li>Signed speed limits exceeding the defaults may apply, and this applies only to the states and mainland territories.</li></ul></td></tr><tr><td rowspan="6"><b>BE</b></td><td rowspan="2">BE-VLG:urban</td><td rowspan="2">50</td><td rowspan="2">BE-VLG:rural</td><td rowspan="2">70</td><td>BE:living_street</td><td>20</td><td rowspan="6"><ul><li>BE-VLG: Flanders</li><li>BE-WAL: Wallonia</li><li>BE-BRU: Brussels-Capital Region</li></ul></td></tr><tr><td><a href="/wiki/Key:cyclestreet" title="Key:cyclestreet">BE:cyclestreet</a></td><td>30</td></tr><tr><td rowspan="2">BE-WAL:urban</td><td rowspan="2">50</td><td rowspan="2">BE-WAL:rural</td><td rowspan="2">90</td><td>BE:zone30</td><td>30</td></tr><tr><td>BE:zone<i>*</i></td><td><i>* (number)</i></td></tr><tr><td rowspan="2">BE-BRU:urban</td><td rowspan="2">30</td><td rowspan="2">BE-BRU:rural</td><td rowspan="2">70</td><td>BE:trunk</td><td>120</td></tr><tr><td>BE:motorway</td><td>120</td></tr><tr><td rowspan="3"><b>BG</b></td><td rowspan="3">BG:urban</td><td rowspan="3">50</td><td rowspan="3">BG:rural</td><td rowspan="3">90</td><td>BG:living_street</td><td>20</td><td rowspan="3"></td></tr><tr><td>BG:trunk</td><td>120</td></tr><tr><td>BG:motorway</td><td>140</td></tr><tr><td rowspan="2"><b>BY</b></td><td rowspan="2">BY:urban</td><td rowspan="2">60</td><td rowspan="2">BY:rural</td><td rowspan="2">90</td><td>BY:living_street</td><td>20</td><td rowspan="2">—</td></tr><tr><td>BY:motorway</td><td>110</td></tr><tr><td rowspan="6"><b>CA</b></td><td>CA-AB:urban</td><td>50 (most roads), 80 (provincial highways)</td><td>CA-AB:rural</td><td>80 (most roads) 100 (provincial highways)</td><td></td><td></td><td rowspan="6"><ul><li>CA-AB: Alberta</li><li>CA-BC: British Columbia</li><li>CA-MB: Manitoba</li><li>CA-ON: Ontario</li><li>CA-QC: Quebec</li><li>CA-SK: Saskatchewan</li></ul><p>Speed limits above the statutory defaults may apply. Most provinces and territories have provisions in their traffic laws allowing speed limits higher than the defaults.</p></td></tr><tr><td>CA-BC:urban</td><td>50</td><td>CA-BC:rural</td><td>80</td><td></td><td></td></tr><tr><td>CA-MB:urban</td><td>50</td><td>CA-MB:rural</td><td>90</td><td></td><td></td></tr><tr><td>CA-ON:urban</td><td>50</td><td>CA-ON:rural</td><td>80</td><td></td><td></td></tr><tr><td>CA-QC:urban</td><td>50</td><td>CA-QC:rural</td><td>80 (paved roads), 70 (unpaved roads)</td><td>CA-QC:motorway</td><td>100</td></tr><tr><td></td><td></td><td></td><td></td><td>CA-SK:nsl</td><td>80 <sup id="cite_ref-1" class="reference"><a href="#cite_note-1">[1]</a></sup></td></tr><tr><td rowspan="2"><b>CH</b></td><td rowspan="2">CH:urban<sup id="cite_ref-2" class="reference"><a href="#cite_note-2">[2]</a></sup></td><td rowspan="2">50</td><td rowspan="2">CH:rural</td><td rowspan="2">80</td><td>CH:trunk</td><td>100</td><td rowspan="2">—</td></tr><tr><td>CH:motorway</td><td>120</td></tr><tr><td rowspan="6"><b>CZ</b></td><td rowspan="6">CZ:urban</td><td rowspan="6">50</td><td rowspan="6">CZ:rural</td><td rowspan="6">90</td><td>CZ:pedestrian_zone</td><td>20</td><td rowspan="6">—</td></tr><tr><td>CZ:living_street</td><td>20</td></tr><tr><td>CZ:urban_motorway</td><td>80</td></tr><tr><td>CZ:urban_trunk</td><td>80</td></tr><tr><td>CZ:trunk</td><td>110</td></tr><tr><td>CZ:motorway</td><td>130</td></tr><tr><td rowspan="3"><b>DE</b></td><td rowspan="3">DE:urban</td><td rowspan="3">50</td><td rowspan="3">DE:rural</td><td rowspan="3">100</td><td>DE:living_street</td><td>7</td><td rowspan="3"><ul><li>There is no speed limit for DE:motorway <sup id="cite_ref-3" class="reference"><a href="#cite_note-3">[3]</a></sup>.</li></ul></td></tr><tr><td>DE:bicycle_road</td><td>30</td></tr><tr><td>DE:motorway</td><td>none</td></tr><tr><td><b>DK</b></td><td>DK:urban</td><td>50</td><td>DK:rural</td><td>80</td><td>DK:motorway</td><td>130</td><td>—</td></tr><tr><td><b>EE</b></td><td>EE:urban</td><td>50</td><td>EE:rural</td><td>90</td><td></td><td></td><td>—</td></tr><tr><td rowspan="4"><b>ES</b></td><td rowspan="4">ES:urban</td><td rowspan="2">50</td><td rowspan="4">ES:rural</td><td rowspan="4">90</td><td>ES:living_street</td><td>20</td><td rowspan="4"><ul><li>Implicit speed in urban areas (<code>ES:urban</code>) can be 50, 30 or 20 depending on the physical properties of the road.</li><li>30km/h on unpaved roads.</li></ul></td></tr><tr><td>ES:zone30</td><td>30</td></tr><tr><td>30</td><td>ES:trunk</td><td>90</td></tr><tr><td>20</td><td>ES:motorway</td><td>120</td></tr><tr><td rowspan="2"><b>FI</b></td><td rowspan="2">FI:urban</td><td rowspan="2">50</td><td rowspan="2">FI:rural</td><td rowspan="2">80</td><td>FI:trunk</td><td>100</td><td rowspan="2">—</td></tr><tr><td>FI:motorway</td><td>120</td></tr><tr><td rowspan="2"><b>FR</b></td><td rowspan="2">FR:urban</td><td rowspan="2">50</td><td rowspan="2">FR:rural</td><td rowspan="2">80</td><td>FR:zone30</td><td>30</td><td rowspan="2">—</td></tr><tr><td>FR:motorway</td><td>130 / 110 (raining)</td></tr><tr><td rowspan="4"><b>GB</b></td><td rowspan="4"></td><td rowspan="4"></td><td rowspan="4"></td><td rowspan="4"></td><td>GB:nsl_restricted</td><td>30 mph</td><td rowspan="4"><table class="wikitable sortable jquery-tablesorter" cellpadding="2" cellspacing="0" border="1"><thead></thead><tbody><tr><td>GB:nsl_single (60 mph)<p>50 mph (vehicles towing caravans or trailers, buses and minibuses up to 12m, HGV)</p></td></tr><tr><td>GB:nsl_dual (70 mph)<p>60 mph (vehicles towing caravans or trailers, buses and minibuses up to 12m, HGV</p></td></tr><tr><td>GB:motorway (70 mph)<p>60 mph (vehicles towing caravans or trailers, HGV)</p></td></tr></tbody><tfoot></tfoot></table><ul><li>This applies to the United Kingdom inclusive of Great Britain as ISO 3166 country codes are preferred.</li><li>In the UK <tt dir="ltr" class="mw-content-ltr" style="background:#EEF;font-size:1em;line-height:1.6"><bdi style="white-space:nowrap"><a href="/wiki/Key:maxspeed:type" title="Key:maxspeed:type">maxspeed:type</a></bdi>=*</tt> is preferred over <tt dir="ltr" class="mw-content-ltr" style="background:#EEF;font-size:1em;line-height:1.6"><bdi style="white-space:nowrap"><a href="/wiki/Key:source:maxspeed" title="Key:source:maxspeed">source:maxspeed</a></bdi>=*</tt></li><li>Measured in miles per hour</li></ul><p><br></p></td></tr><tr><td>GB:motorway</td><td>70 mph</td></tr><tr><td>GB:nsl_dual</td><td>70 mph</td></tr><tr><td>GB:nsl_single</td><td>60 mph</td></tr><tr><td rowspan="2"><b>GR</b></td><td rowspan="2">GR:urban</td><td rowspan="2">50</td><td rowspan="2">GR:rural</td><td rowspan="2">90</td><td>GR:trunk</td><td>110</td><td rowspan="2">—</td></tr><tr><td>GR:motorway</td><td>130</td></tr><tr><td rowspan="3"><b>HU</b></td><td rowspan="3">HU:urban</td><td rowspan="3">50</td><td rowspan="3">HU:rural</td><td rowspan="3">90</td><td>HU:living_street</td><td>20</td><td rowspan="3">—</td></tr><tr><td>HU:trunk</td><td>110</td></tr><tr><td>HU:motorway</td><td>130</td></tr><tr><td rowspan="2"><b>IT</b></td><td rowspan="2">IT:urban</td><td rowspan="2">50</td><td rowspan="2">IT:rural</td><td rowspan="2">90</td><td>IT:trunk</td><td>110</td><td rowspan="2">—</td></tr><tr><td>IT:motorway</td><td>130</td></tr><tr><td rowspan="2"><b>JP</b></td><td rowspan="2"></td><td rowspan="2"></td><td rowspan="2"></td><td rowspan="2"></td><td>JP:nsl</td><td>60</td><td rowspan="2">—</td></tr><tr><td>JP:express</td><td>100</td></tr><tr><td><b>LT</b></td><td>LT:urban</td><td>50</td><td>LT:rural</td><td>90</td><td></td><td></td><td>—</td></tr><tr><td><b>NO</b></td><td>NO:urban</td><td>50</td><td>NO:rural</td><td>80</td><td></td><td></td><td>—</td></tr><tr><td><b>PH</b></td><td>PH:urban</td><td>30</td><td>PH:rural</td><td>80</td><td>PH:express</td><td>100</td><td><ul><li>Buses and trucks subject to lower speed limits (80 km/h in motorways<sup id="cite_ref-PH-Motorways_4-0" class="reference"><a href="#cite_note-PH-Motorways-4">[4]</a></sup>, 50 km/h in rural areas)</li><li>Cars and motorcycles in urban areas subject to higher speed limits only in designated through streets or boulevards without blind corners (40 km/h).</li><li>Places classified as "crowded streets" by local government units, areas close to intersections at blind corners and school zones have a speed limit of 20 km/h regardless of motor vehicle type.</li><li>Local government units may assign lower speed limits. The Toll Regulatory Board sets the speed limits on expressways <sup id="cite_ref-5" class="reference"><a href="#cite_note-5">[5]</a></sup>; the general speed limit for expressway is set at 100 (80 for heavy trucks, buses and any vehicle hauling a trailer) unless otherwise posted.</li></ul></td></tr><tr><td rowspan="2"><b>PT</b></td><td rowspan="2">PT:urban</td><td rowspan="2">50</td><td rowspan="2">PT:rural</td><td rowspan="2">90</td><td>PT:trunk</td><td>100</td><td rowspan="2">—</td></tr><tr><td>PT:motorway</td><td>120</td></tr><tr><td rowspan="2"><b>RO</b></td><td rowspan="2">RO:urban</td><td rowspan="2">50</td><td rowspan="2">RO:rural</td><td rowspan="2">90</td><td>RO:trunk</td><td>100</td><td rowspan="2">—</td></tr><tr><td>RO:motorway</td><td>130</td></tr><tr><td rowspan="3"><b>RS</b></td><td rowspan="3">RS:urban</td><td rowspan="3">50</td><td rowspan="3">RS:rural</td><td rowspan="3">80</td><td>RS:living_street</td><td>10</td><td rowspan="3">—</td></tr><tr><td>RS:trunk</td><td>100</td></tr><tr><td>RS:motorway</td><td>130</td></tr><tr><td rowspan="2"><b>RU</b></td><td rowspan="2">RU:urban</td><td rowspan="2">60</td><td rowspan="2">RU:rural</td><td rowspan="2">90</td><td>RU:living_street</td><td>20</td><td rowspan="2">—</td></tr><tr><td>RU:motorway</td><td>110</td></tr><tr><td rowspan="2"><b>SE</b></td><td rowspan="2">SE:urban</td><td rowspan="2">50</td><td rowspan="2">SE:rural</td><td rowspan="2">70</td><td>SE:trunk</td><td>90</td><td rowspan="2">—</td></tr><tr><td>SE:motorway</td><td>110</td></tr><tr><td rowspan="2"><b>SI</b></td><td rowspan="2">SI:urban</td><td rowspan="2">50</td><td rowspan="2">SI:rural</td><td rowspan="2">90</td><td>SI:trunk</td><td>110</td><td rowspan="2">—</td></tr><tr><td>SI:motorway</td><td>130</td></tr><tr><td rowspan="4"><b>SK</b></td><td rowspan="4">SK:urban</td><td rowspan="4">50</td><td rowspan="4">SK:rural</td><td rowspan="4">90</td><td>SK:living_street</td><td>20</td><td rowspan="4">—</td></tr><tr><td>SK:trunk</td><td>90</td></tr><tr><td>SK:motorway</td><td>130</td></tr><tr><td>SK:motorway_urban</td><td>90</td></tr><tr><td rowspan="3"><b>UA</b></td><td rowspan="3">UA:urban</td><td rowspan="3">50</td><td rowspan="3">UA:rural</td><td rowspan="3">90</td><td>UA:living_street</td><td>20</td><td rowspan="3">—</td></tr><tr><td>UA:trunk</td><td>110</td></tr><tr><td>UA:motorway</td><td>130</td></tr><tr><td rowspan="3"><b>UK</b></td><td rowspan="3"></td><td rowspan="3"></td><td rowspan="3"></td><td rowspan="3"></td><td><s>UK:motorway</s></td><td><s>70 mph</s></td><td rowspan="3"><ul><li>Use of "GB:" terminology is preferred as ISO 3166 country codes are used.</li><li>Measured in miles per hour.</li></ul></td></tr><tr><td><s>UK:nsl_dual</s></td><td><s>70 mph</s></td></tr><tr><td><s>UK:nsl_single</s></td><td><s>60 mph</s></td></tr><tr><td rowspan="2"><b>UZ</b></td><td rowspan="2">UZ:urban</td><td rowspan="2">70</td><td rowspan="2">UZ:rural</td><td rowspan="2">100</td><td>UZ:living_street</td><td>30</td><td rowspan="2">—</td></tr><tr><td>UZ:motorway</td><td>110</td></tr></tbody>'''

In [53]:
soup = BeautifulSoup(html)
tds = soup.find_all("td")
d = {}
for t in zip(tds[:-1],tds[1:]):
    country_code, maxspeed = t[0].get_text(), t[1].get_text()
    if not re.match("\w\w:[\w\d:]*$", country_code):
        continue

    try:
        if not maxspeed.endswith("mph"):
            maxspeed = str(int(maxspeed))
    except:
        maxspeed = "None"
    # print(country_code, maxspeed)
    d[country_code] = maxspeed

print(d)

{'AR:urban': '40', 'AR:rural': '110', 'AR:urban:primary': '60', 'AR:urban:secondary': '60', 'AT:urban': '50', 'AT:rural': '100', 'AT:bicycle_road': '30', 'AT:trunk': '100', 'AT:motorway': '130', 'AU:urban': 'None', 'AU:rural': 'None', 'BE:living_street': '20', 'BE:cyclestreet': '30', 'BE:zone30': '30', 'BE:trunk': '120', 'BE:motorway': '120', 'BG:urban': '50', 'BG:rural': '90', 'BG:living_street': '20', 'BG:trunk': '120', 'BG:motorway': '140', 'BY:urban': '60', 'BY:rural': '90', 'BY:living_street': '20', 'BY:motorway': '110', 'CH:rural': '80', 'CH:trunk': '100', 'CH:motorway': '120', 'CZ:urban': '50', 'CZ:rural': '90', 'CZ:pedestrian_zone': '20', 'CZ:living_street': '20', 'CZ:urban_motorway': '80', 'CZ:urban_trunk': '80', 'CZ:trunk': '110', 'CZ:motorway': '130', 'DE:urban': '50', 'DE:rural': '100', 'DE:living_street': '7', 'DE:bicycle_road': '30', 'DE:motorway': 'None', 'DK:urban': '50', 'DK:rural': '80', 'DK:motorway': '130', 'EE:urban': '50', 'EE:rural': '90', 'ES:urban': '50', 'ES:r